In [193]:
from helper_functions import xor_hex_strings, single_byte_xor, repeating_key_xor, xor_bytes, hamming_distance
from file_loading import load_file_as_bytes, load_file_as_b64
from conversions import int2bytes, hex2bytes, string2bytes, hex2b64
from symmetric_encryption import is_ecb_mode, aes128_ecb_encrypt, aes128_ecb_decrypt, aes128_cbc_encrypt, aes128_cbc_decrypt, determine_block_size
from padding import apply_pkcs_7, remove_pkcs_7
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 9: Implement PKCS#7 padding**

In [194]:
apply_pkcs_7(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## **Challenge 10: Implement CBC mode**

In [195]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)
text = b'YELLOW SUBMARINE YELLOW SUBMARIN'

aes128_cbc_decrypt(aes128_cbc_encrypt(text, key, iv), key, iv)

b'YELLOW SUBMARINE YELLOW SUBMARIN'

In [196]:
file_bytes = load_file_as_b64("challenge_data/10.txt", remove_newlines=True)

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
aes128_cbc_decrypt(file_bytes, key, IV).decode('utf-8')

"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take a

## **Challenge 11: An ECB/CBC detection oracle**

In [197]:
def encryption_oracle_11(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in either ECB or CBC mode. The plaintext is padded with PKCS#7 padding
    and a random prefix and suffix are added to the plaintext. The key and IV are chosen at random. The function
    returns a tuple of the mode used and the cyphertext. 
    
    `encrypt(before_bytes + plaintext + after_bytes)`
    """

    key = secrets.token_bytes(16)  # for AES-128
    iv = secrets.token_bytes(16)  # for AES-128

    before_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))
    after_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))

    modified_plaintext = apply_pkcs_7(before_bytes + plaintext + after_bytes, 16)

    if secrets.choice([True, False]):
        print("ECB")
        return aes128_ecb_encrypt(modified_plaintext, key)
    else:
        print("CBC")
        return aes128_cbc_encrypt(modified_plaintext, key, iv)

In [198]:
for _ in range(5):
    ecb_mode = is_ecb_mode(encryption_oracle_11, 16)
    if ecb_mode:
        print("ECB", "\n")
    else:
        print("CBC", "\n")

ECB
ECB 

ECB
ECB 

ECB
ECB 

ECB
ECB 

CBC
CBC 



## **Challenge 12: Byte-at-a-time ECB decryption (Simple)**

In [199]:
def encryption_oracle_12(plaintext: bytes) -> bytes:
    unknown_string = base64.decodebytes(b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    key = secrets.token_bytes(16)
    return aes128_ecb_encrypt(apply_pkcs_7(plaintext + unknown_string, 16), key)

In [209]:
# step 1: find the block length
block_size, offset = determine_block_size(encryption_oracle_12)
print(f"Block length: {block_size}")
print(f"Padding offset (bytes to add so that there is no padding): {offset}")

# step 2: determine if the encryption is ECB or CBC
print(f"Is this encrypted with ECB mode: {is_ecb_mode(encryption_oracle_12, block_size)}")

# step 3: make dictionary of all possible last bytes by brute forcing the block by block
plaintext = b''
for char_i in range(10000):
    cypher2plain = {}
    input_str = b''

    block_num = char_i // block_size # current block number

    prev_decryption = plaintext[(1-block_size):]
    first_block_padding = b'A' * (block_size - len(prev_decryption) - 1)
    for i in range(256):
        # last byte will be brute forced
        input_str += first_block_padding + prev_decryption + int2bytes(i) 
    
    input_str += b'A' * ((block_size - len(plaintext) - 1) % block_size) # for the last block
    cyphertext = encryption_oracle_12(input_str)

    for i in range(256):
        cypher = cyphertext[i*block_size:(i+1)*block_size]
        cypher2plain[cypher] = int2bytes(i)

    next_block_num = i + 1 + block_num
    test_cypher = cyphertext[(next_block_num)*block_size:(next_block_num + 1)*block_size]

    try:
        plaintext += cypher2plain[test_cypher]
    except Exception:
        break

plaintext.decode('utf-8')

Block length: 16
Padding offset (bytes to add so that there is no padding): 10
Is this encrypted with ECB mode: True


"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"